# Tutorial 06: Quantum Espresso Wrapper
In this tutorial we will go through a simple example of how to use the wrapper for the Quantum Espresso simulation engine. You can find the wrapper [here](https://github.com/simphony/quantum-espresso-wrapper).
## Background
This is an example of a slightly different design based upon the input-output functionality of certain simulation engines such as Quantum Espresso and Gromacs.


## Let's get hands-on
### Installation
To run the local installation of Quantum Espresso, simply run `./install_engine.sh`. This should check for the prerequisites and compile the code for Quantum Espresso for you.

If the script runs into an error finding openmpi-bin or something like that, try running `apt-get update` and try again. 
Once the installation has completed, try running `pw.x` to see if the installation has succeeded. If this does not work, then try adding `export PATH=$PATH:/home/username/qe-6.5/bin/` at the end of `.bashrc` located at your home folder.  
  
Once you have verified that `pw.x` works, install the ontology via `pico install ontology.simlammps.yml`, and make sure to run `python3 setup.py` located in the root of the quantum espresso wrapper folder.   

Note that the installation of the ontology requires version 3.4.0 of osp-core to be installed (see [dev branch](https://github.com/simphony/osp-core/tree/v3.4.0-dev))

That should be all needed to use Quantum Espresso!

### Simple example

This is an adaptation of quantum-espresso-wrapper/examples/Simple.py. As usual, we need to import the necessary components:

In [1]:
import numpy as np 

from osp.core.namespaces import QE
from osp.core.utils import pretty_print
from osp.wrappers.quantumespresso.qe_session import qeSession

Next, we create simulation and its K points, which determine at what points it samples the cell

In [2]:
sim = QE.Simulation()
k = QE.K_POINTS(vector6 = (7, 7, 7, 0, 0, 0), unit = "")


Next, we create a cell, the element Silicon, a pseudopotential, an atom and the cell parameters. Note that the pseudopotential files should ALWAYS be located inside of a folder named `$PSEUDO_DIR` inside of wherever you are running the simulation.

In [3]:
SiCell = QE.Cell()
Si = QE.Element(name = "Si")
SiPseudo = QE.PSEUDOPOTENTIAL(name = "Si.pbe-n-kjpaw_psl.1.0.0.UPF")
Si1 = QE.Atom()
SiParams = QE.CellParams()
celldm1 = QE.Celldm1(value = 5.43070, unit = "au")

Next, we connect these all to each other using the `add` method.

In [4]:
Si.add(SiPseudo, Si1)
Si.add(QE.Mass(value = 28.085, unit = "amu"))
SiCell.add(Si1, SiParams)
Si1.add(QE.Position(vector = (0, 0, 0), unit = ""))
SiCell.add(celldm1)

<qe.Celldm1: e076d0c6-8934-4d96-aeea-546b83e26c89,  CoreSession: @0x7f83fd5f7340>

We specify the cell parameters:

In [5]:
SiParams.add(QE.CellParameterX(vector = (0.5, 0.5, 0), unit = ""),
             QE.CellParameterY(vector = (0.5, 0, 0.5), unit = ""),
             QE.CellParameterZ(vector = (0, 0.5, 0.5), unit = ""))

[<qe.CellParameterX: 3ec36c5e-4c46-4823-9667-f1dc5a610520,  CoreSession: @0x7f83fd5f7340>,
 <qe.CellParameterY: 2f393d38-a72d-452b-800f-c1f8ce11482a,  CoreSession: @0x7f83fd5f7340>,
 <qe.CellParameterZ: 7059b956-4c70-4c9e-8434-a8d57c08cbe2,  CoreSession: @0x7f83fd5f7340>]

And then we add everything created so far to the simulation:


In [6]:
sim.add(SiCell)
sim.add(Si)
sim.add(k)

<qe.K_POINTS: 462dc0bf-35a1-4083-89ea-354fc38eee48,  CoreSession: @0x7f83fd5f7340>

While we're add it, let's add some variables to the simulation which we can check to see if they have been updated. They will not be taken into account when simulating, so they're there for control purposes.


In [7]:
sim.add(QE.Pressure(value = 100, unit = "kbar"))
sim.add(QE.StressTensor(tensor2 = np.zeros((3, 3)), unit = "kbar"))

<qe.StressTensor: 09f0f304-f6fc-4faa-81e2-bbdc6dada874,  CoreSession: @0x7f83fd5f7340>

Let's check out what this simulation looks like now with the `pretty_print` function:

In [8]:
pretty_print(sim)

- Cuds object:
  uuid: 900ad868-56f9-4b3f-90a2-d1f6b08c6876
  type: qe.Simulation
  superclasses: cuba.Class, cuba.Entity, qe.Simulation
  description: 
    All components of the simulation that are needed to run the model

   |_Relationship qe.HAS_PART:
     -  qe.Cell cuds object:
     .  uuid: 55f048cb-3f21-44d4-a152-29855ae71ffc
     .   |_Relationship qe.HAS_PART:
     .     -  qe.Atom cuds object:
     .     .  uuid: 74f35152-188d-4446-91ac-b5be017a6f73
     .     .   |_Relationship qe.HAS_PART:
     .     .     -  qe.Position cuds object:
     .     .        uuid: 9752a94e-626b-4696-92fd-ccad8c4f6761
     .     .        vector: [0. 0. 0.]
     .     .        unit: 
     .     -  qe.CellParams cuds object:
     .     .  uuid: 6028eebd-dede-499c-a41e-aca7636c38de
     .     .   |_Relationship qe.HAS_PART:
     .     .     -  qe.CellParameterX cuds object:
     .     .     .  uuid: 3ec36c5e-4c46-4823-9667-f1dc5a610520
     .     .     .  vector: [0.5 0.5 0. ]
     .     .     .  un

Now, it's time to get the simulation running:

In [9]:
session = qeSession()
quantum_espresso_wrapper = QE.QEWrapper(session = session)
quantum_espresso_wrapper.add(sim)
print("Running calculation...")

quantum_espresso_wrapper.session._run(simulation = sim, prefix = "si", command_type = "pw.x", calculation_type = "scf", root = "", CONTROL = {'pseudo_dir': "'.'"})

Running calculation...
si.pwscf.in
pw.x -i si.pwscf.in > si.pwscf.out


Now let's check the results of our calculation:

In [10]:
pretty_print(sim)

- Cuds object:
  uuid: 900ad868-56f9-4b3f-90a2-d1f6b08c6876
  type: qe.Simulation
  superclasses: cuba.Class, cuba.Entity, qe.Simulation
  description: 
    All components of the simulation that are needed to run the model

   |_Relationship qe.HAS_PART:
     -  qe.Cell cuds object:
     .  uuid: 55f048cb-3f21-44d4-a152-29855ae71ffc
     .   |_Relationship qe.HAS_PART:
     .     -  qe.Atom cuds object:
     .     .  uuid: 74f35152-188d-4446-91ac-b5be017a6f73
     .     .   |_Relationship qe.HAS_PART:
     .     .     -  qe.Force cuds object:
     .     .     .  uuid: 3dea8718-f430-4be9-8986-d97a7ffef349
     .     .     .  vector: [0. 0. 0.]
     .     .     .  unit: N
     .     .     -  qe.Position cuds object:
     .     .        uuid: 9752a94e-626b-4696-92fd-ccad8c4f6761
     .     .        vector: [0. 0. 0.]
     .     .        unit: 
     .     -  qe.CellParams cuds object:
     .     .  uuid: 6028eebd-dede-499c-a41e-aca7636c38de
     .     .   |_Relationship qe.HAS_PART:
     .

As you can see, the original part of the cuds tree is still there, with everything mostly the same. The new parts are:

- The qe.PwOut cuds object. This is the output file of the simulation, in case there is something that the wrapper does not parse but that you would still like to see.
- The qe.TotalEnergy cuds object. This was parsed from the qe.PwOut file itself.
- The qe.Force cuds object. This represents the force exerted on the atom(s).

The updated parts are:

- The qe.Pressure cuds object, having changed in value from 100 kbar to 5723.64 kbar.
- The qe.StressTensor cuds object, which is no longer zero.

Let's see if we can do better and calculate some bands structures:

In [11]:
quantum_espresso_wrapper.session._run(simulation = sim, prefix = "si", command_type = "pw.x", calculation_type = "bands")
quantum_espresso_wrapper.session._run(simulation = sim, prefix = "si", command_type = "bands.x", calculation_type = "")

si.pwbands.in
pw.x -i si.pwbands.in > si.pwbands.out
si.bands.in
bands.x -i si.bands.in > si.bands.out


Although the cuds structure won't have changed much by this, the data is there in the folder.

Now let's try to relax this cell. While it isn't a real cell, we can still perform the calculations to relax it to know what the movement of the atoms would be like if it were a real cell (warning, perform vc-relax type calculations with caution. These examples are designed to be lightweight and non-indicative of real-world applications).

In [12]:
quantum_espresso_wrapper.session._run(simulation = sim, prefix = "si", command_type = "pw.x", calculation_type = "relax", IONS = {'ion_dynamics': "'bfgs'"})

si.pwrelax.in
pw.x -i si.pwrelax.in > si.pwrelax.out


In [13]:
pretty_print(sim)

- Cuds object:
  uuid: 900ad868-56f9-4b3f-90a2-d1f6b08c6876
  type: qe.Simulation
  superclasses: cuba.Class, cuba.Entity, qe.Simulation
  description: 
    All components of the simulation that are needed to run the model

   |_Relationship qe.HAS_PART:
     -  qe.BandsDat cuds object:
     .  uuid: 9ae8f7ea-c044-435e-a8cd-6e017a66cae2
     .  path: si.bands.dat
     -  qe.Cell cuds object:
     .  uuid: 55f048cb-3f21-44d4-a152-29855ae71ffc
     .   |_Relationship qe.HAS_PART:
     .     -  qe.Atom cuds object:
     .     .  uuid: 74f35152-188d-4446-91ac-b5be017a6f73
     .     .   |_Relationship qe.HAS_PART:
     .     .     -  qe.Force cuds object:
     .     .     .  uuid: 3dea8718-f430-4be9-8986-d97a7ffef349
     .     .     .  vector: [0. 0. 0.]
     .     .     .  unit: N
     .     .     -  qe.Position cuds object:
     .     .        uuid: 9752a94e-626b-4696-92fd-ccad8c4f6761
     .     .        vector: [0. 0. 0.]
     .     .        unit: kbar
     .     -  qe.CellParams cuds

In this example, the position hasn't changed, since there's only one atom. So let's spice things up a little bit and add another atom, and then relax:

In [14]:
Si2 = QE.Atom()
Si2.add(QE.Position(vector = (0.25, 0.25, 0.26), unit = ""))
SiCell.add(Si)
Si.add(Si2)
pretty_print(sim)

- Cuds object:
  uuid: 900ad868-56f9-4b3f-90a2-d1f6b08c6876
  type: qe.Simulation
  superclasses: cuba.Class, cuba.Entity, qe.Simulation
  description: 
    All components of the simulation that are needed to run the model

   |_Relationship qe.HAS_PART:
     -  qe.BandsDat cuds object:
     .  uuid: 9ae8f7ea-c044-435e-a8cd-6e017a66cae2
     .  path: si.bands.dat
     -  qe.Cell cuds object:
     .  uuid: 55f048cb-3f21-44d4-a152-29855ae71ffc
     .   |_Relationship qe.HAS_PART:
     .     -  qe.Atom cuds object:
     .     .  uuid: 74f35152-188d-4446-91ac-b5be017a6f73
     .     .   |_Relationship qe.HAS_PART:
     .     .     -  qe.Force cuds object:
     .     .     .  uuid: 3dea8718-f430-4be9-8986-d97a7ffef349
     .     .     .  vector: [0. 0. 0.]
     .     .     .  unit: N
     .     .     -  qe.Position cuds object:
     .     .        uuid: 9752a94e-626b-4696-92fd-ccad8c4f6761
     .     .        vector: [0. 0. 0.]
     .     .        unit: kbar
     .     -  qe.CellParams cuds

In [15]:
quantum_espresso_wrapper.session._run(simulation = sim, prefix = "si", command_type = "pw.x", calculation_type = "relax", IONS = {'ion_dynamics': "'bfgs'"})
pretty_print(sim)

si.pwrelax.in
pw.x -i si.pwrelax.in > si.pwrelax.out
- Cuds object:
  uuid: 900ad868-56f9-4b3f-90a2-d1f6b08c6876
  type: qe.Simulation
  superclasses: cuba.Class, cuba.Entity, qe.Simulation
  description: 
    All components of the simulation that are needed to run the model

   |_Relationship qe.HAS_PART:
     -  qe.BandsDat cuds object:
     .  uuid: 9ae8f7ea-c044-435e-a8cd-6e017a66cae2
     .  path: si.bands.dat
     -  qe.Cell cuds object:
     .  uuid: 55f048cb-3f21-44d4-a152-29855ae71ffc
     .   |_Relationship qe.HAS_PART:
     .     -  qe.Atom cuds object:
     .     .  uuid: 74f35152-188d-4446-91ac-b5be017a6f73
     .     .   |_Relationship qe.HAS_PART:
     .     .     -  qe.Force cuds object:
     .     .     .  uuid: 3dea8718-f430-4be9-8986-d97a7ffef349
     .     .     .  vector: [ 4.2290e-05 -6.8198e-04 -6.8198e-04]
     .     .     .  unit: N
     .     .     -  qe.Position cuds object:
     .     .        uuid: 9752a94e-626b-4696-92fd-ccad8c4f6761
     .     .        ve